In [163]:
import re
import itertools

import numpy as np
import pandas as pd

from tqdm import tqdm

import mph
from tools import plots

import matplotlib.pyplot as plt
import plotly.express as px
import peewee

In [2]:
client = mph.start()

In [119]:
model = client.load('models/Kinetic model.mph')

In [4]:
mph.tree(model)

Parametric search
├─ parameters
│  ├─ Rate_constants
│  ├─ Initial_concentrations
│  ├─ Molecular_masses
│  └─ Technical
├─ functions
├─ components
│  └─ Chemical_reactions
├─ geometries
├─ views
├─ selections
├─ coordinates
├─ variables
├─ couplings
├─ physics
│  ├─ Reaction Engineering
│  │  ├─ Initial Values 
│  │  ├─ Species: Q
│  │  ├─ Species: DH
│  │  ├─ Species: Qm
│  │  ├─ Species: DHp
│  │  ├─ Species: QH
│  │  ├─ Species: D
│  │  ├─ Species: QHD
│  │  ├─ Species: QHH
│  │  ├─ Species: prod
│  │  ├─ Species: QD
│  │  ├─ Ke: Q + DH => Qm + DHp
│  │  ├─ KH: Qm + DHp => QH + D
│  │  ├─ Kr: QH + D => QHD
│  │  ├─ Kp: QHD =>QHH + prod
│  │  ├─ KqH//Kdisp: Q + QHH <=> 2 QH
│  │  ├─ Kph: Q => prod
│  │  ├─ Ks: Q + QHD =>QH + QD
│  │  ├─ Kd//Kc: QD <=>Q + D
│  │  ├─ KrD: 2 D => prod
│  │  └─ Estimation
│  └─ Sensitivity
│     ├─ Global Objective 1
│     └─ Global Objective 2
├─ multiphysics
├─ materials
├─ meshes
├─ studies
│  └─ Generator
│     ├─ Sensitivity
│     └─ Time Dependent

In [175]:
client.clear()

In [ ]:
def sweep_to_sql(
    model: mph.Model,
    name,
    desc=None,
):
    _, light_sweep = model.outer('Data')
    i = 1
    params = {
        key: float(value)
        for key,
        value in _comsol.model_parameters(model).items()
        if 'light' not in key
    }
    params['name'] = name
    params['desc'] = desc

    notes = []
    for light_value in light_sweep:
        note = {}
        note.update(params)
        note['light'] = light_value
        df = _comsol.evaluate_expressions(model, i)
        note['data'] = df.to_json(index=True)
        notes.append(note)

        i += 1
    return


In [ ]:
def compare(from_node: mph.Node, to_node: mph.Node):
    from_properties, to_properties = from_node.properties(), to_node.properties()
    all_properties = sorted(set(from_properties).union(set(to_properties)))

    print(f"{'from ' + from_node.name(): >53} | {'to '+to_node.name(): <53}")
    for key in all_properties:
        from_prop = str(from_properties.get(key, None))
        to_prop = str(to_properties.get(key, None))

        string = f'{"*" if (from_prop!=to_prop) else " "} {key: <30} '+\
                 f'{from_prop[:20]: <20} | {to_prop[:20]: <20}'
        print(string)


def copy_settings(from_node: mph.Node, to_node: mph.Node):
    for i in range(2):
        from_properties, to_properties = from_node.properties(), to_node.properties()

        auto_settings = {}
        for key, value in from_properties.items():
            if (str(value) == 'auto') or (to_properties.get(key, None) is None):
                auto_settings.update({key: value})
                continue

            to_node.property(name=key, value=value)

        for key, value in auto_settings.items():
            if to_properties.get(key, None) is None: continue
            to_node.property(name=key, value=value)


def copy_solver(from_solver: mph.Node, to_solver: mph.Node, verbose=False):
    copy_settings(from_solver, to_solver)

    from_dict = {node.name(): node for node in from_solver.children()}
    to_dict = {node.name(): node for node in to_solver.children()}

    for node in from_dict:
        from_node = from_dict[node]
        to_node = to_dict[node]
        copy_settings(from_node, to_node)
        if verbose:
            compare(from_node, to_node)
            print('*' * 120)


def input_check(string):
    if string is None:
        while string != 'q':
            string = input(f'Set {string=}, to quit - q:')
            string = string.strip()
    return string

In [107]:
class AbstractStudy:
    study_name = None

    def __init__(self, model: mph.Model):
        self.comsol_model = model

    @property
    def time_end(self):
        return self.comsol_model.parameters()['time_end']

        
    @property
    def constants(self):
        rule = lambda key: key[0] == 'K'
        return {
            key: value
            for key, value in self.comsol_model.parameters().items()
            if rule(key)
        } #yapf: disable

    @property
    def initial(self):
        rule = lambda key: ('0' in key) or (key in ['light'])
        return {
            key: value
            for key, value in self.comsol_model.parameters().items()
            if rule(key)
        } #yapf: disable

    @property
    def species(self) -> dict:
        reaction_node_children = [
            i.name() for i in self.nodes['reaction'].children()
        ]

        species = re.findall(
            string='\n'.join(reaction_node_children),
            pattern='Species: (.*)',
        )
        return {specie: f'reaction.c_{specie}' for specie in species}

    @property
    def nodes(self):
        study_node = self.comsol_model / 'studies' / f'{self.study_name}'
        assert study_node.exists(), f'Study node does not exist'

        solution_node = self.comsol_model / 'solutions' / f'{self.study_name}_Solution'
        assert study_node.exists(), f'Solution node does not exist'

        data_node = self.comsol_model / 'datasets' / f'{self.study_name}_data'
        assert study_node.exists(), f'Data node does not exist'

        reaction_node = model / 'physics' / 'Reaction Engineering'
        assert study_node.exists(), f'Reaction node does not exist'

        nodes_dict = {
            'study': study_node,
            'solution': solution_node,
            'data': data_node,
            'reaction': reaction_node
        }
        return nodes_dict

    def set_parametrs(self, **parameters):
        for key, value in parameters.items():
            self.comsol_model.parameter(name=key, value=value)

    @staticmethod
    def _set_node_properties(node: mph.Node, **properties):
        for key, value in properties.items():
            node.property(key, value)

    def temporal_eval(
        self,
        functions: dict,
        outer_number=1,
    ) -> pd.DataFrame:

        model = self.comsol_model
        functions.update({'time': 't'})
        row_data = model.evaluate(
            list(functions.values()),
            dataset=self.nodes['data'].name(),
            outer=outer_number,
        )
        return pd.DataFrame(row_data, columns=list(functions))

    def solve(self):
        self.comsol_model.solve(study=self.nodes['study'].name())


In [7]:
class Generator(AbstractStudy):
    study_name = 'Generator'

    def temporal_eval(self, functions={}) -> pd.DataFrame:
        return super().temporal_eval(outer_number=1, functions=functions)


In [8]:
def make_comdinations(diap: dict):
    keys = list(diap.keys())
    values = [diap[key] for key in keys]
    combinations = list(itertools.product(*values))
    result = [dict(zip(keys, comb)) for comb in combinations]
    return result


def sweep(combinations, generator: Generator):

    result = []
    combinations = tqdm(iterable=combinations)
    for combination in combinations:
        generator.set_parametrs(**combination)
        generator.solve()
        df = generator.temporal_eval(generator.species())
        df.loc[:, combination.keys()] = list(combination.values())
        result.append(df)
    return pd.concat(result)



In [170]:
class Sensitivity(AbstractStudy):
    study_name = 'Sensivity'

    @property
    def sensitivities(self):
        return {key: f'fsens({key})' for key in self.constants}

    @property
    def nodes(self):
        nodes_dict = super().nodes

        sensivity_node = nodes_dict['study'] / 'Sensitivity'
        assert sensivity_node.exists(), f'Estimation node does not exist'

        nodes_dict.update({'sensitivity': sensivity_node})
        return nodes_dict

    @property
    def constants(self):
        all_parameters = self.nodes['sensitivity'].properties()
        filtered_properties = {
            key: value
            for key, value
            in zip(all_parameters['pname'], all_parameters['initval'])
        } #yapf: disable

        rule = lambda key: key[0] == 'K'
        return {
            key: value
            for key, value in filtered_properties.items()
            if rule(key)
        } #yapf: disable

    @property
    def initial(self):
        all_parameters = self.nodes['sensitivity'].properties()
        filtered_properties = {
            key: value
            for key, value
            in zip(all_parameters['pname'], all_parameters['initval'])
        } #yapf: disable

        rule = lambda key: ('0' in key) or (key in ['light'])
        return {
            key: value
            for key, value
            in filtered_properties.items()
            if rule(key)
        } #yapf: disable

    def set_parametrs(self, **parameters):
        all_parameters = self.constants
        all_parameters.update(self.initial)
        old_len = len(all_parameters)

        all_parameters.update(parameters)
        assert len(all_parameters) == old_len, 'Parametrs not exist'

        self.nodes['sensitivity'].property(
            name='pname',
            value=list(all_parameters),
        )
        self.nodes['sensitivity'].property(
            name='initval',
            value=[str(i) for i in all_parameters.values()],
        )

    @property
    def target(self):
        result = self.nodes['sensitivity'].properties()['optobj'][0]
        return result.replace('comp.reaction.c_', '')

    def set_target(self, target: str):
        assert f'reaction.c_{target}' in self.species.values(), 'Target is not specie'
        self.nodes['sensitivity'].property(
            name='optobj',
            value=[f'comp.reaction.c_{target}'],
        )


In [14]:
# TODO: out of found parametrs
class Estimator(AbstractStudy):
    study_name = 'Estimator'

    @property
    def nodes(self):
        nodes_dict = super().nodes
        
        estimation_node = self.comsol_model / 'physics' / 'Reaction Engineering' / 'Estimation'
        assert estimation_node.exists(), f'Estimation node does not exist'
        
        nodes_dict.update({'estimation': estimation_node})
        return nodes_dict

    @property
    def experiments(self):
        return self.nodes['estimation_node'].children()

    @property
    def tables(self):
        tables = self.comsol_model / 'tables'
        experiment_tables = [
            node for node in tables.children() if 'Experiment' in node.name()
        ]
        return experiment_tables

    def create_one_experiment(
        self,
        data,
        data_columns,
        experiment_i,
        path=r'D:\WORKS\COMSOL_polymers\Batch\generator_out_short.csv',
    ):
        experiment_name = f'exp{experiment_i}'

        # create experiment
        self.nodes['estimation_node'].java.create(
            experiment_name,
            "Experiment",
            -1,
        )
        experiment = self.experiments[-1]

        # create table
        table_tag = f"tbl_compreactionest1{experiment_name}"
        table = (self.comsol_mode / 'tables').java.create(table_tag, "Table")
        table.label(f"Experiment {experiment_i} Table")
        table.setTableData(data)
        table.active(False)

        # set up parametrs
        variables_dict = {'Time': 't'}
        variables_dict.update(self.species())
        variables = [variables_dict[key] for key in data_columns]

        Estimator._set_node_properties(
            node=experiment,
            fileName=path,
            dataColumn=data_columns,
            use=[1] * len(data_columns),
            modelVariable=variables,
        )

    def create_experiments(self, datas: list[pd.DataFrame]):
        i = 0
        for data in datas:
            self.create_one_experiment(
                data=data,
                data_columns=data.columns,
                experiment_i=i,
                path=r'./generator_out_short.csv',
            )
            i += 1

    def clear_experiments(self):
        experiments, tables = self.experiments, self.tables
        for table in tables:
            table.remove()
        for experiment in experiments:
            experiment.remove()

    def solve(self):
        self.nodes['estimation'].toggle('on')
        try:
            self.solve()
        finally:
            self.nodes['estimation'].toggle('off')

# Generator

In [153]:
gen = Generator(model)

In [159]:
gen.solve()

In [160]:
species = {
    key: value
    for key,
    value in gen.species.items()
    if key in ['Q', 'DH', 'QHH']
}
df = gen.temporal_eval(species)

In [161]:
df

,Q,DH,QHH,time
0,1000.000000,1000.000000,0.000000e+00,0.000000e+00
1,999.980470,999.980470,0.000000e+00,1.953125e-10
2,999.965955,999.965955,8.553503e-18,3.404698e-10
3,999.936927,999.936927,5.756430e-16,6.307845e-10
4,999.878878,999.878878,5.764835e-14,1.211414e-09
...,...,...,...,...
744,328.488721,0.501034,3.280325e+02,1.970925e-04
745,328.477270,0.489041,3.280332e+02,1.978300e-04
746,328.466074,0.477334,3.280339e+02,1.985675e-04
747,328.455128,0.465908,3.280345e+02,1.993049e-04


In [164]:
df = df/1000
df['time'] = df['time']*1000

In [165]:
def simple_temporal_plot(df:pd.DataFrame):
    fig = px.line(df, x='time',y=[col for col in df.columns if col not in ['time','light']])
    fig.update_layout(
        height=500,
        margin={
            'r': 0, 'l': 0, 't': 0, 'b': 0
        },
        legend=dict(x=-0.1, y=1, xanchor="center"),
    )
    return fig

In [167]:
simple_temporal_plot(df)


# Sweep

In [ ]:
gen.constants

In [36]:
a=make_comdinations({
    'light':(np.linspace(0.1, 5.1, 11)/1000).round(7)
})

In [41]:
df= sweep(a,gen)

100%|██████████| 11/11 [00:14<00:00,  1.34s/it]


In [42]:
df

,Q,DH,Qm,DHp,QH,D,QHD,QHH,prod,QD,time,light
0,1000.000000,1.000000e+03,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0001
1,999.980470,9.999805e+02,0.019530,0.019530,7.449999e-08,7.449999e-08,0.000000e+00,0.000000e+00,1.953087e-12,1.455078e-17,1.953125e-10,0.0001
2,999.965955,9.999660e+02,0.034045,0.034045,2.427463e-07,2.427463e-07,8.553506e-18,8.553503e-18,3.404619e-12,4.978598e-17,3.404698e-10,0.0001
3,999.936927,9.999369e+02,0.063072,0.063072,1.397627e-06,1.397627e-06,5.756434e-16,5.756430e-16,6.308150e-12,4.555122e-16,6.307845e-10,0.0001
4,999.878878,9.998789e+02,0.121112,0.121112,9.914367e-06,9.914367e-06,5.764842e-14,5.764835e-14,1.217081e-11,6.211388e-15,1.211414e-09,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...
2017,332.761279,-3.099783e-103,0.005256,0.005256,4.707437e-01,6.002663e-05,3.339151e+02,3.328045e+02,3.330182e+02,4.312137e-02,1.940677e-04,0.0051
2018,332.761076,8.850607e-104,0.005206,0.005206,4.707438e-01,5.888321e-05,3.339148e+02,3.328047e+02,3.330182e+02,4.352527e-02,1.958852e-04,0.0051
2019,332.760874,7.241769e-104,0.005157,0.005157,4.707438e-01,5.777216e-05,3.339144e+02,3.328049e+02,3.330182e+02,4.392918e-02,1.977028e-04,0.0051
2020,332.760671,2.193951e-104,0.005109,0.005109,4.707438e-01,5.669227e-05,3.339141e+02,3.328051e+02,3.330182e+02,4.433308e-02,1.995203e-04,0.0051


# Sensitivity

In [171]:
sens = Sensitivity(model)

{'Kc': 'fsens(Kc)',
 'Kd': 'fsens(Kd)',
 'Kdisp': 'fsens(Kdisp)',
 'Ke': 'fsens(Ke)',
 'KH': 'fsens(KH)',
 'Kp': 'fsens(Kp)',
 'Kph': 'fsens(Kph)',
 'KqH': 'fsens(KqH)',
 'Kr': 'fsens(Kr)',
 'KrD': 'fsens(KrD)',
 'Ks': 'fsens(Ks)'}

In [149]:
sens.set_target('QHH')

In [111]:
sens.set_parametrs(Kdisp=1e9)

In [151]:
sens.solve()

# Plots

In [ ]:
import plotly.express as px

In [ ]:
rule = {'Kc': (0, 0.30000)}
diap = ['Kc', 'light']
datas, dfs = get_solves(rule)
result = collect_dfs(datas, dfs, diap)


In [ ]:
df_gen

In [ ]:
result =df_gen
fig = px.line(
    result,
    x="time",
    y="DH",
    animation_frame='light',
    # color='Kc',
    range_y=[0, 1],
)
fig.update_layout(
    height=500,
    margin={
        'r': 0, 'l': 0, 't': 0, 'b': 0
    },
    legend=dict(x=-0.1, y=1, xanchor="left"),
)

# Estimator

In [ ]:
# est =Estimator(model)